In [ ]:
!pip install langchain-community


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:
!pip install pypdf
from langchain.document_loaders import PyPDFLoader

raw_doc=PyPDFLoader("paracetamol.pdf")
raw_doc


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:
raw_load=raw_doc.load()
raw_load

[Document(metadata={'source': 'paracetamol.pdf', 'page': 0}, page_content=' 1\nPackage leaflet: Information for the user \n \nParacetamol 500 mg tablets \nparacetamol  \n \n \nRead all of this leaflet carefully before you start taking this  medicine because it contains \nimportant information for you. \nAlways take this medicine exactly as described in this leaflet or as your doctor or pharmacist told you. \n- Keep this leaflet. You may need to read it again. \n- Ask your pharmacist if you need more information or advice. \n- If you get any side effects, talk to your doctor, or pharmacist . This includes any possible side \neffects not listed in this leaflet. See section 4. \nYou must talk to a doctor if you do not feel better or if you feel worse after 3 days of fever and 5 \ndays of pain or if other symptoms appear.   \n \nWhat is in this leaflet  \n1. What Paracetamol is and what it is used for \n2. What you need to know before you take Paracetamol \n3. How to take Paracetamol \n4. 

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitted_text=RecursiveCharacterTextSplitter(chunk_size=100,chunk_overlap=20)
splitted_text

In [ ]:
#Chunks of text
text_split=splitted_text.split_documents(raw_load)
text_split

In [ ]:
!pip install sentence_transformers


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/tmp/ipykernel_1299/1378607185.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

In [ ]:
!pip install chromadb


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Create Chroma vector store
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
db = Chroma.from_documents(documents=text_split, embedding=embedding_model)
db

In [ ]:
pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1 autoawq==0.2.6 autoawq-kernels==0.0.7

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, AwqConfig

In [ ]:
model_id = "hugging-quants/Meta-Llama-3.1-8B-Instruct-AWQ-INT4"

quantization_config = AwqConfig(
    bits=4,
    fuse_max_seq_len=512, # Note: Update this as per your use-case
    do_fuse=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
  model_id,
  torch_dtype=torch.float16,
  low_cpu_mem_usage=True,
  device_map="auto",
  quantization_config=quantization_config
)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/295 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:186: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.However, loading attributes (e.g. ['version', 'fuse_max_seq_len', 'exllama_config', 'modules_to_fuse', 'do_fuse']) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.
  warnings.warn(warning_msg)


model.safetensors.index.json:   0%|          | 0.00/63.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.68G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
retriever=db.as_retriever()
question="what age group can have paracetamol?"
context_docs=retriever.invoke(question)
context_docs

[Document(metadata={'page': 0, 'source': 'paracetamol.pdf'}, page_content='Paracetamol is indicated in adults and adolescents aged 12 or older.'),
 Document(metadata={'page': 2, 'source': 'paracetamol.pdf'}, page_content='Paracetamol is not recommended for children under 12 years of age. Other formulations containing'),
 Document(metadata={'page': 1, 'source': 'paracetamol.pdf'}, page_content='Paracetamol is not recommended for children under 12 years of age. Other formulations containing'),
 Document(metadata={'page': 2, 'source': 'paracetamol.pdf'}, page_content='Normal adult dosage of paracetamol is usually appropriate. However, in frail, immobile elderly')]

In [ ]:
from langchain.prompts import PromptTemplate

# Define the prompt_template variable
prompt_template = """Use the following context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

{question}
"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

In [ ]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
from langchain_core.output_parsers import StrOutputParser

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=150,
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

/tmp/ipykernel_1299/4271996020.py:17: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


In [ ]:
from langchain_core.output_parsers import StrOutputParser

In [ ]:
llm_chain = prompt | llm | StrOutputParser()
llm_chain

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="Use the following context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\n{question}\n")
| HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x75c1dce57520>)
| StrOutputParser()

In [ ]:
question = "what age group can have paracetamol?"
context = " ".join([doc.page_content for doc in context_docs])
tokens = tokenizer(question, context, return_tensors='pt')

llm_chain.invoke({"context":context, "question": question})

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


'Use the following context to answer the question at the end. If you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer.\n\nParacetamol is indicated in adults and adolescents aged 12 or older. Paracetamol is not recommended for children under 12 years of age. Other formulations containing Paracetamol is not recommended for children under 12 years of age. Other formulations containing Normal adult dosage of paracetamol is usually appropriate. However, in frail, immobile elderly\n\nwhat age group can have paracetamol?\nA) Children under 12\nB) Adults and adolescents over 12\nC) Frail, immobile elderly\nD) All of the above\nAnswer: B) Adults and adolescents over 12\nExplanation: According to the text, "Paracetamol is indicated in adults and adolescents aged 12 or older." Therefore, the correct answer is B) Adults and adolescents over 12. Note that this answer choice only includes individuals who are 12 or older, which matches the indication for paraceta